# Algorithme EM

In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

C:\Users\srotc_000\Anaconda3\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [78]:
wine = pd.read_csv('Datasets/Debug.csv', delimiter = ";")

In [79]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.array(wine.ix[:,d+2:]) #labels des annotateurs
x = np.array(wine.ix[:,0:d]) #variables explicatives
w0 = np.random.rand(1,d)

## Initialisation

In [64]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

## Etape E

In [65]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    CE = 0 #Conditionnal excepectation
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    for i in range(0,N):
        CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
    return mu


## Maximum Log - Likelihood Estimator

In [66]:
def logLikelihood(w, y, alpha, beta):
    a = ai(alpha, y)
    b = bi(beta, y)
    p = pi(x, w)
    
    #On calcule directement la log-vraissemblance.
    vraissemblance = 0
    for i in range(0,N):
        vraissemblance = vraissemblance + np.log((a[i]*p[i])+b[i]*(1-p[i]))
        
    return vraissemblance

## Étape M

In [67]:
#M-step

def alpha_function(mu,y):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append(tmp1/tmp2)
	return alpha

def beta_function(mu,y):
    beta = []
    for j in range(0,R):
        tmp1 = 0
        tmp2 = 0
        for i in range(0,N):
            tmp1 += (1-mu[i])*(1-y[i][j])
            tmp2 += 1-mu[i]
        beta.append(tmp1/tmp2)
    return beta

def updateW(w,x, eta, mu):
    g = 0
    for i in range(0,N):
        g += (mu[i] - sigma(x[i].dot(w.T)))*x[i]
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    w = w - eta*np.linalg.inv(H).dot(g)
    return w

In [68]:
updateW(w0,x,0.1,mu0)

array([[ -2.91664074e+14,   5.85459645e+14,  -1.43714984e+15,
          2.39286939e+14,   2.69461175e+15,  -1.10821101e+14,
         -1.38676250e+14,   1.05663039e+16,  -1.79510183e+15,
          3.45043812e+15,  -3.12714069e+14]])

## Itérations

In [80]:
mu = init_mu(y)
alpha = alpha_function(mu,y)
beta = beta_function(mu,y)
diff_w = 10
w = w0

compteur = 0

#while (np.linalg.norm(diff_w) > 0.001) : # Limite de convergence à decider
while (compteur < 1000):
    mu = E_step(x,y,alpha,beta,w)
    alpha = alpha_function(mu,y)
    beta = beta_function(mu,y)
    w_bis = updateW(w,x,0.01,mu)
    diff_w = w - w_bis
    w = w_bis
    if (compteur % 100 == 0):
        print ("ITERATION : ", compteur)
        print("Vraissemblance : ", logLikelihood(w, y, alpha, beta))
        print("Norme de diff_w : ", np.linalg.norm(diff_w))
        print("Alpha : ", alpha)
        print("Beta : ", beta)
    compteur = compteur + 1

ITERATION :  0
Vraissemblance :  [-426.09887498]
Norme de diff_w :  68.0876286538
Alpha :  [array([ 0.48780494]), array([ 0.48780494]), array([ 0.48780494]), array([ 0.48780494]), array([ 0.51219506])]
Beta :  [array([ 0.99999993]), array([ 0.99999993]), array([ 0.99999993]), array([ 0.99999993]), array([  6.54365045e-08])]
ITERATION :  100
Vraissemblance :  [-103.1283375]
Norme de diff_w :  0.0534869700799
Alpha :  [array([ 1.]), array([ 1.]), array([ 1.]), array([ 1.]), array([ 0.])]
Beta :  [array([ 1.]), array([ 1.]), array([ 1.]), array([ 1.]), array([ 0.])]
ITERATION :  200
Vraissemblance :  [-62.35489096]
Norme de diff_w :  0.063887036333
Alpha :  [array([ 1.]), array([ 1.]), array([ 1.]), array([ 1.]), array([ 0.])]
Beta :  [array([ 1.]), array([ 1.]), array([ 1.]), array([ 1.]), array([ 0.])]
ITERATION :  300
Vraissemblance :  [-55.29080202]
Norme de diff_w :  0.0454041196576
Alpha :  [array([ 1.]), array([ 1.]), array([ 1.]), array([ 1.]), array([ 0.])]
Beta :  [array([ 1.]),

In [154]:
class model:
    """Classe qui encapsule l'apprentissage"""
    
    def __init__(self, maxIter = 10000, eta = 0.01):
        """Constructeur"""
        self.maxIter = maxIter
        self.eta = eta
        
    def initMu(self, y):
        """Initialisation de mu"""
        mu = []
        for i in range(0,N):
            mu.append(np.sum(y[i])/R)
        return mu
    
    def ai(self, alpha, y):
        """Update du vecteur a (1xN)"""
        a = []
        for i in range(0,N):
            proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
            a.append(proda)
        self.a = a
        
    def bi(self, beta, y):
        """Update du vecteur b (1xN)"""
        b = []
        for i in range(0,N):
            prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
            b.append(prodb)
        self.b = b
        
    def pi(self, x, w):
        """Update du vecteur p (1xN)"""
        p = []
        for i in range(0,N):
            p.append(sigma(x[i].dot(w.T)))
        self.p = p
        
    def mui(self, a,b,p):
        """Update du vecteur mu (1xN). C'est l'étape E."""
        mu = []
        for i in range(0,N):
            mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
        self.mu = mu        

In [16]:
np.sum(x)

10470.80069